# Data preparation

In [ ]:
# import pandas to work with our data
# import NumPy
# import matplotlib to plot charts and Seaborn to enhance charts

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid")

from datetime import datetime

# fix random seed for reproducibility
np.random.seed(7)

hours_per_week = 168 # equals 7*24

In [ ]:
# read data from csv
data_set = pd.read_csv('C:/Users/ahutz/OneDrive/Dokumente/ML/Daten/Energy/time_series_energy.csv')

# preview data
data_set.head()

In [ ]:
# check if the data types (or dtypes) have been correctly interpreted
data_set.dtypes

In [ ]:
# convert timestamp to datetime format
data_set['timestamp'] = pd.to_datetime(data_set['timestamp'])
data_set.dtypes

In [ ]:
# set timestamp as index for easy access to dataframe
data_set.index = data_set['timestamp']

In [ ]:
# count missing values
data_set.isnull().sum()

In [ ]:
# interpolate missing values
data_set['price_day_ahead'].interpolate(method='linear', inplace=True)
data_set['load_forecast'].interpolate(method='linear', inplace=True)
data_set.isnull().sum()

# Data Exploration

In [ ]:
# descriptive statistics for features
data_set.describe()

In [ ]:
#visualize time series data
plt.figure(figsize = (12,8))
sns.lineplot(x="timestamp", y="price_day_ahead", data=data_set['2018'])
plt.show()

In [ ]:
#visualize multiple time series as line plots
data_plot = data_set['2018-05']
f, axes = plt.subplots(3, 1, sharey=False, figsize=(18, 10))
sns.lineplot(x="timestamp", y="price_day_ahead", data=data_plot, ax = axes[0])
sns.lineplot(x="timestamp", y="load_forecast", data=data_plot, ax = axes[1])
sns.lineplot(x="timestamp", y="wind_forecast", data=data_plot, ax = axes[2])
plt.show()

In [ ]:
#visualize the distribution of the feature variables
f, axes = plt.subplots(1, 3, sharey=False, figsize=(18, 6))
sns.distplot(data_set['price_day_ahead'], kde=False, color="b", ax = axes[0])
sns.distplot(data_set['load_forecast'], kde=False, color="b", ax = axes[1])
sns.distplot(data_set['wind_forecast'], kde=False, color="b", ax = axes[2])
plt.show()

In [ ]:
#Visualize the relation between variables 
#Scatter plots show how much one variable is affected by another
#The relationship between two variables is called their correlation
#negative vs positive correlation
sns.pairplot(data_set['2018'][['load_forecast','wind_forecast','price_day_ahead']],plot_kws=dict(alpha=0.75))

In [ ]:
# calculate correlation matrix
data_set.corr()

# Basic feature generation

In [ ]:
# extract values from timestamps
data_set['year'] = data_set['timestamp'].dt.year
data_set['month_of_year'] = data_set['timestamp'].dt.month
data_set['day_of_month'] = data_set['timestamp'].dt.day
data_set['day_of_week'] = data_set['timestamp'].dt.weekday
data_set['hour_of_day'] = data_set['timestamp'].dt.hour
data_set.head()

In [ ]:
# calculate additional feature
data_set['price_last_week'] = data_set['price_day_ahead'].shift(hours_per_week)
data_set.head()

In [ ]:
# delete rows with nan values 
df = data_set.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [ ]:
# calculate correlation matrix
df.corr()

# Split into training and testing data 

In [ ]:
# make copy of original
df_original = df.copy()

# use 3 years of data for training 
# use data week 1 2019 for testing
train = df.ix['2016-01-08':'2019-01-07']
test = df.ix['2019-01-07':'2019-01-13']

# Benchmark estimator

In [ ]:
# as benchmark we assume the electricity spot price last week and this week are the same (naive approach)
y_hat = test.copy()
y_hat['naive'] = test['price_last_week']

In [ ]:
# visualize actual prices together with predicted prices
plt.figure(figsize = (12,6))
sns.lineplot(x='timestamp', y='price_day_ahead', label = 'Test', data = test[:'2019-01-13'])
sns.lineplot(x='timestamp', y='naive', label = 'Naive', data = y_hat[:'2019-01-13'])
plt.show()

In [ ]:
# visualize actual prices versus with predicted prices
sns.scatterplot(x = test['price_day_ahead'], y = y_hat['naive'])

### Calculate root mean square error as metric for benchmark accuracy

In [ ]:
# compare predicted values to target
# calculate difference between prediction and target, 
# then take square of difference (because differences can be positive or negative), 
# then divide the sum of all values by the number of observations (mean)
# to get same unit of measurement of error as target data, take square root
# smaller value means that predicted values are close to target values and vice versa
from sklearn.metrics import mean_squared_error
from math import sqrt

rmse_naive = sqrt(mean_squared_error(test['price_day_ahead'], y_hat['naive']))
print('RMSE Naive Test : %.3f' % rmse_naive)

# Neural network estimator

In [ ]:
# Keras is high-level neural networks API, written in Python and capable of running on top of TensorFlow
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# copy & split the data
df_ann = df_original.copy()
train_nn = df_ann.ix['2016-01-08':'2019-01-07']
test_nn = df_ann.ix['2019-01-07':'2019-01-13']

# separate into feature set and target variable
train_nn, test_nn = train_nn.drop(['timestamp'],1), test_nn.drop(['timestamp'],1)
trainX, testX = train_nn.drop(['price_day_ahead'], 1), test_nn.drop(['price_day_ahead'], 1)
trainY, testY = train_nn['price_day_ahead'], test_nn['price_day_ahead']

trainX.to_numpy()
testX.to_numpy()
trainY.to_numpy()
testY.to_numpy()

In [ ]:
# define structure of neural network, a feedforward neural network
# input layer has 8 input nodes, one for each input feature (load forecast, wind forecast, time attributes and price_last_week)
# three hidden layers with 12, 100 and 100 nodes, respectively
# output layer has 1 output node, the predicted electricity spot price
model = Sequential()
model.add(Dense(12,input_dim=8,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(100,activation='relu'))
model.add(Dense(1,activation='linear'))

# configures the model for training
# we use adam which uses Stochastic Gradient Descent and mse as loss function
model.compile(loss='mean_squared_error', optimizer='adam')

# summarizes the neural network structure
model.summary()

In [ ]:
# train neural network model
# network is trained for a total of 100 epochs, meaning that the model “sees” each individual training example 100 times
# model parameters are updated using batches of size 100, meaning the number of examples per weight update
model.fit(trainX, trainY, epochs=100, batch_size=100, verbose=0, shuffle=False)

In [ ]:
# returns the loss value for the model
train_mse = model.evaluate(trainX, trainY, verbose=0)
test_mse = model.evaluate(testX, testY, verbose=0)

# make predictions
trainPredict_nn = model.predict(trainX)
testPredict_nn = model.predict(testX)

In [ ]:
#visualize the predictions and target values on timeline
y_hat['nn'] = testPredict_nn.reshape(-1)

plt.figure(figsize = (12,6))
sns.lineplot(x='timestamp', y='price_day_ahead', label = 'Test', data = test[:'2019-01-11'])
sns.lineplot(x='timestamp', y='naive', label = 'Naive', data = y_hat[:'2019-01-11'])
sns.lineplot(x='timestamp', y='nn', label = 'NN', data = y_hat[:'2019-01-11'])
plt.show()

In [ ]:
# visualize actual prices versus with predicted prices
f, axes = plt.subplots(1, 2, sharey=False, figsize=(15, 6))
sns.scatterplot(x="price_day_ahead", y="naive", data = y_hat[:'2019-01-11'], ax = axes[0])
sns.scatterplot(x="price_day_ahead", y="nn", data = y_hat[:'2019-01-11'], ax = axes[1])
plt.show()

In [ ]:
#calculate rmse
rmse_nn = sqrt(mean_squared_error(testY, testPredict_nn))
print('RMSE NN Test : %.3f' % rmse_nn)
print('RMSE Naive Test : %.3f' % rmse_naive)

## Determine importance of features

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
import eli5
from eli5.sklearn import PermutationImportance

def base_model():
    model = Sequential()        
    model.add(Dense(12,input_dim=9,activation='relu'))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(1,activation='linear'))

    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

X = trainX
y = trainY

my_model = KerasRegressor(build_fn=base_model, epochs=100, batch_size=168, verbose=0)    
my_model.fit(X,y)

perm = PermutationImportance(my_model, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names = X.columns.tolist())

## Optimize hyperparameters neural network

In [ ]:
# Use scikit-learn to grid search the batch size and epochs
from sklearn.model_selection import GridSearchCV
# create model
model_kr = KerasRegressor(build_fn=base_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model_kr, param_grid=param_grid, n_jobs=-1, cv=3)
X = trainX
y = trainY
grid_result = grid.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))